In [1]:
import numpy as np
from ansys.mapdl.core import launch_mapdl
mapdl = launch_mapdl()
mapdl.finish()
mapdl.clear()
print(mapdl)  # show ANSYS version and PyMAPDL version

# --------------------------------------------------------------------------
# ------------------------------ parameters --------------------------------
# --------------------------------------------------------------------------
# units SI [Pa, N, m, Nm]
# E_beam = 70e9
# pr_beam = 0.0
# width = 0.1
# height = 0.02
# depth = 0.01
# M = 2000
# F = 3000

# validation guide data VM16
E_beam = 30e6  # psi
pr_beam = 0.0
width = 10  # inch
height = 2
depth = 1
M = 2000  # in-lb (cas1; target deflection = 0.005 [in], stress = 3000 [psi])
F = 300  # lb  (case2; target deflection = 0.005, stress = 4050))
# plane182 (plane183?)
# FORMULAS FOR STRESS AND STRAIN, ROARK, 4TH ED.,PG. 104,106

mapdl.prep7()

# USING PLANE42 ELEMENTS
mapdl.antype("static")
mapdl.et(1, "plane182")
mapdl.keyopt(1, 1, 3)  # IMPLIFIED ENHANCED STRAIN FORMULATION
mapdl.mp("ex", 1, E_beam)
mapdl.mp("nuxy", 1, pr_beam)

n_bl = mapdl.n(1)
n_br = mapdl.n(6, x=10)
mapdl.fill()
mapdl.ngen(3, 10, node1=n_bl, node2=n_br, ninc=1, dy=1)
e0 = mapdl.e(1, 2, 12, 11)
e1 = mapdl.e(11, 12, 22, 21)
mapdl.egen(5, ninc=1, iel1=e0, iel2=e1)
#mapdl.open_gui()
# VM16, CASE 1 - END MOMENT, ROARK, PAGE 106, NO. 9
mapdl.d(1, "all", nend=21, ninc=10)
# mapdl.f(6, "fx", 1000)
# mapdl.f(26, "fx", -1000)
mapdl.f(16, "mx", 2000)
mapdl.outpr("nsol", 1)
mapdl.outpr("esol", 1)
mapdl.finish()
mapdl.run("/solu")
mapdl.output("SCRATCH")
mapdl.solve()
mapdl.post1()
mapdl.set("last")
mapdl.graphics("power")
mapdl.eshape(scale=1)
mapdl.view(1, 1, 1, 1)
mapdl.output()
bend_stress1 = mapdl.result.nodal_stress(0)[1].max()
u1 = mapdl.get_value("node", 26, "u", "y")

mapdl.exit()
# FINI
# /SOLU
# /COM, VM16, CASE 2 - END LOAD, ROARK, PAGE 104, NO. 1
# F,6,FX,,,16,10
# F,6,FY,150,,16,10
# /OUT,SCRATCH
# SOLVE
# FINISH
# /POST1
# SET,LAST
# /GRAPHICS,POWER
# /ESHAPE,1
# /VIEW,1,1,1,1
# /OUT,
# *GET,U2,NODE,16,U,Y
# PLNSOL,S,X !BENDING STRESS
# *GET,BEND_STRESS2,PLNSOL,0,MAX
# *DIM,LABEL,CHAR,2,2
# *DIM,VALUE,,2,3
# *DIM,VALUE2,,2,3
# LABEL(1,1) = 'DEFL','BEND_STRESS'
# LABEL(1,2) = ' (in) ',' (psi) '
# *VFILL,VALUE(1,1),DATA,.00500,3000
# *VFILL,VALUE(1,2),DATA,U1,BEND_STRESS1
# *VFILL,VALUE(1,3),DATA,ABS(U1/.005),ABS(BEND_STRESS1/3000)
# SAVE,TABLE_3
# *VFILL,VALUE2(1,1),DATA,.00500,4050
# *VFILL,VALUE2(1,2),DATA,U2,BEND_STRESS2
# *VFILL,VALUE2(1,3),DATA,ABS(U2/.005),ABS(BEND_STRESS2/4050)
# SAVE,TABLE_4

Product:         ANSYS Academic Research
MAPDL Version:   RELEASE  Release 19.1      BUILD 19.1      UPDATE 20180418
PyMAPDL Version: 0.58.8

